In [1]:
import pandas as pd
import numpy as np
import PyPDF2 as pdf
import os
from tqdm import tqdm
import re
import json
import collections


# pdfminer.six
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
import pdfplumber

In [2]:
df = pd.read_excel('頁碼簿_分段狀況(測試).xlsx', dtype='str')

In [3]:
df[['startpage', 'endpage']] = df[['startpage', 'endpage']].astype(int)
df = df[df['Classified']=='可部分拆分']
df = df.head(10)
df = df.iloc[:,[0,1,2,3,4,5,6,7,8]]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 1 to 16
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   co_id            10 non-null     object
 1   year             10 non-null     object
 2   ext              10 non-null     object
 3   filename         10 non-null     object
 4   isedible         10 non-null     object
 5   startpage        10 non-null     int32 
 6   endpage          10 non-null     int32 
 7   inputpath        10 non-null     object
 8   filename_folder  10 non-null     object
dtypes: int32(2), object(7)
memory usage: 720.0+ bytes


In [5]:
df

,co_id,year,ext,filename,isedible,startpage,endpage,inputpath,filename_folder
1,1102.0,2020,pdf,2020_1102_20210625F04_20211212_154258.pdf,True,5,11,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1102/20...,2020_1102_20210625F04_20211212_154258.pdf
2,1103.0,2019,pdf,2019_1103_20200622F04_20211212_155216.pdf,True,4,5,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1103/20...,2019_1103_20200622F04_20211212_155216.pdf
4,1108.0,2019,pdf,2019_1108_20200618F04_20211212_155632.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1108/20...,2019_1108_20200618F04_20211212_155632.pdf
5,1108.0,2020,pdf,2020_1108_20210616F04_20211212_155819.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1108/20...,2020_1108_20210616F04_20211212_155819.pdf
6,1109.0,2019,pdf,2019_1109_20200623F04_20211212_155859.pdf,True,5,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1109/20...,2019_1109_20200623F04_20211212_155859.pdf
7,1109.0,2020,pdf,2020_1109_20210624F04_20211212_155958.pdf,True,5,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1109/20...,2020_1109_20210624F04_20211212_155958.pdf
8,1110.0,2019,pdf,2019_1110_20200620F04_20211212_160051.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1110/20...,2019_1110_20200620F04_20211212_160051.pdf
9,1110.0,2020,pdf,2020_1110_20210625F04_20211212_160217.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1110/20...,2020_1110_20210625F04_20211212_160217.pdf
15,1216.0,2019,pdf,2019_1216_20200624F04_20211212_163421.pdf,True,6,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1200食品工業\1216/20...,2019_1216_20200624F04_20211212_163421.pdf
16,1216.0,2020,pdf,2020_1216_20210623F04_20211212_163450.pdf,True,6,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1200食品工業\1216/20...,2020_1216_20210623F04_20211212_163450.pdf


In [6]:
def extract_text_by_page(pdf_path,start,end, maxpages=0, pagenos=None): # extract whole document
    with pdfplumber.open(pdf_path) as fh:
        text_by_page = ''
        for i in range(start,end):
            text = fh.pages[i]
            text_by_page += text.extract_text()
            text_by_page = text_by_page.replace('\n','')
    fh.close()
    return text_by_page #ok

In [7]:
extract_text_by_page(df['inputpath'][1],df['startpage'][1]-1,df['endpage'][1]-1)

'致股東報告書   致股東報告書 壹、2020 年國際暨國內經濟情勢回顧                                                                                 2020年國際經濟情勢回顧     受COVID-19新型冠狀病毒肺炎疫情影響，2020年世界各國普遍採取高度的防疫封鎖措施，導致全球經濟成長動能大幅減緩，根據中華經濟研究院分析，受創最重的產業，以航空、旅遊、餐飲、旅館、以及娛樂業為主，雖然物流、電子商務、以及遠距通訊等產業因需求大增而受惠，但全球經濟還是陷入 1930年代大蕭條以來的最大衰退。     許多國家儘管採取寬鬆貨幣政策，來緩和衝擊，但成效有限，歐美等疫情嚴重國家，均出現數十年來經濟成長最大跌幅；部分新興市場國家，例如印度、墨西哥……，因為疫情嚴峻，經濟萎縮幅度更超過 7.5%，反倒是防疫有成的越南、中國大陸及臺灣等亞洲國家，在刺激內需政策，和外需擴張助益下，內部經濟有 2%以上的正成長。     根據 IHS Markit 分析，去年全球經濟成長率為-3.9%，是2010 年以來首度負成長；國際貨幣基金 IMF所發布的世界經濟展望報告，2020年全球經濟成長率為-3.3％，為 10 年來最低。 2020年國內經濟情勢回顧     2020年COVID-19疫情席捲全球，臺灣 2020年代表字「疫」字獲得最多名人推薦，顯示疫情翻轉了大眾生活，也改變了全世界經濟格局。     各國為了防止疫情擴散，紛紛採取封城、管制邊境、限制人員活動等嚴格措施，導致各類活動與需求幾乎停擺，受到國際經濟衰退拖累，臺灣上半年景氣也明顯下滑；所幸政府和國民齊心協力，防疫有成，生產、製造、消費等活動得以持續，各項振興方案效果逐漸發揮成效，維持一定程度消費。     臺灣經濟研究院分析，第三季起，各主要國家經濟逐步重啟，國內傳統產業景氣，隨著海外需求回升而復甦，電子資訊產業受惠於遠距商機需求，半導體大廠接獲歐美新訂單，以及中國大陸廠商在美方制裁下急單湧現，讓臺灣下半年製造業景氣逐漸轉強。    行政院主計總處公布的 2020年臺灣經濟成長率為 3.11%，是自 1991年來，首度超越中國大陸，更居已開發國家之冠。 2020 年公司營業表現 一、2020 年中國大陸整體水泥消耗量約 23.77 億公噸，

# 測試__________________________________________

In [13]:
def aaa(pdf_file,start,end):
    
    texts = extract_text_by_page(pdf_file,start,end)
    paragraph = dict()
    
    loc = {'1':-1, '2':-1, '3':-1, '4':-1}
    text=seq=exception=score=fseq=''
  
    for t in texts:
        text+=t
#     print(text)
    
    # 列舉全部情況
    paragraph['sit1'] = sit1loc(text)
    paragraph['sit2'] = sit2loc(text)
    paragraph['sit3'] = sit3loc(text)
    paragraph['sit4'] = sit4loc(text)
    
    return paragraph

In [90]:
aaa(df['inputpath'][1],df['startpage'][1]-1,df['endpage'][1]-1)

{'sit1': [<re.Match object; span=(1555, 1557), match='營業'>,
  [<re.Match object; span=(2, 4), match='收入'>],
  <re.Match object; span=(6578, 6580), match='營業'>,
  [<re.Match object; span=(2, 4), match='收入'>],
  <re.Match object; span=(6553, 6555), match='營運'>,
  [<re.Match object; span=(2, 4), match='狀況'>],
  <re.Match object; span=(1822, 1823), match='年'>,
  [<re.Match object; span=(3, 5), match='績效'>],
  <re.Match object; span=(1823, 1825), match='經營'>,
  [<re.Match object; span=(2, 4), match='績效'>]],
 'sit2': [<re.Match object; span=(6381, 6383), match='營運'>,
  [<re.Match object; span=(2, 4), match='計畫'>],
  <re.Match object; span=(6376, 6377), match='年'>,
  [<re.Match object; span=(7, 9), match='計畫'>]],
 'sit3': [<re.Match object; span=(5803, 5805), match='未來'>,
  [<re.Match object; span=(8, 10), match='發展'>],
  <re.Match object; span=(5803, 5805), match='未來'>,
  [<re.Match object; span=(10, 12), match='方向'>]],
 'sit4': []}

In [136]:
paragraph = aaa(df['inputpath'][1],df['startpage'][1]-1,df['endpage'][1]-1)

In [137]:
type(paragraph)

dict

In [138]:
seq =  sequence_speculate(dump_paragraph(paragraph))
print(seq)

2341


In [139]:
dump_paragraph(paragraph)

{'sit1': [{'key': '營業', 'start': 1555, 'end': 1557},
  [{'key': '收入', 'start': 2, 'end': 4}],
  {'key': '營業', 'start': 6578, 'end': 6580},
  [{'key': '收入', 'start': 2, 'end': 4}],
  {'key': '營運', 'start': 6553, 'end': 6555},
  [{'key': '狀況', 'start': 2, 'end': 4}],
  {'key': '年', 'start': 1822, 'end': 1823},
  [{'key': '績效', 'start': 3, 'end': 5}],
  {'key': '經營', 'start': 1823, 'end': 1825},
  [{'key': '績效', 'start': 2, 'end': 4}]],
 'sit2': [{'key': '營運', 'start': 6381, 'end': 6383},
  [{'key': '計畫', 'start': 2, 'end': 4}],
  {'key': '年', 'start': 6376, 'end': 6377},
  [{'key': '計畫', 'start': 7, 'end': 9}]],
 'sit3': [{'key': '未來', 'start': 5803, 'end': 5805},
  [{'key': '發展', 'start': 8, 'end': 10}],
  {'key': '未來', 'start': 5803, 'end': 5805},
  [{'key': '方向', 'start': 10, 'end': 12}]],
 'sit4': []}

In [140]:
test = dump_paragraph(paragraph)

In [151]:
test['sit4'] = "[{'key': '未來', 'start': 5803, 'end': 5805},[{'key': '發展', 'start': 8, 'end': 10}]"

In [145]:
test['sit3']

[{'key': '未來', 'start': 5803, 'end': 5805},
 [{'key': '發展', 'start': 8, 'end': 10}],
 {'key': '未來', 'start': 5803, 'end': 5805},
 [{'key': '方向', 'start': 10, 'end': 12}]]

In [147]:
test['sit4']

"[{'key': '未來', 'start': 5803, 'end': 5805},[{'key': '發展', 'start': 8, 'end': 10}]"

In [148]:
cut_paragraph_by_sequence(test,sequence=seq)

TypeError: string indices must be integers

# 測試結束________________________

In [7]:
def cut_paragraph(pdf_file,start,end):
    
    texts = extract_text_by_page(pdf_file,start,end)
    paragraph = dict()
    
    loc = {'1':-1, '2':-1, '3':-1, '4':-1}
    text=seq=exception=score=fseq=''
  
    for t in texts:
        text+=t
#     print(text)
    
    # 列舉全部情況
    paragraph['sit1'] = sit1loc(text)
    paragraph['sit2'] = sit2loc(text)
    paragraph['sit3'] = sit3loc(text)
    paragraph['sit4'] = sit4loc(text)
    
    # classify letter type
    classification_type, score = letter_classification(paragraph)
    
    #debug
#     print(score)
#     print(classification_type)
#     print(paragraph)
#     print(text)
#     return loc, paragraph, score, seq, fseq, text, exception

    # 預測段落順序
    seq = sequence_speculate(dump_paragraph(paragraph))
    loc, exception = cut_paragraph_by_sequence(dump_paragraph(paragraph),sequence=seq)
    fseq=''
    for item in sorted(loc.items(),key=lambda t:t[1], reverse=False):
        fseq+=item[0]

    # if paragraph too short, exception
    if location_dist_too_short(loc,len(text), fseq):
        exception+='~~dist too short'
    if score<4:
        exception = '~~some situation not found'
        
    
    return loc, paragraph, score, seq, fseq, text, exception


In [8]:
def sit1loc(line):
    kw1 = ['營業計劃','營業計畫','營運計劃','營運計畫', '營業', '營運', '年', '經營']
    kw2 = ['實施成果','結果', '成果', '報告書', '概況', '狀況', '情形', '績效', '回顧']
    matches = list()
    interval = 5
    # 只找後面
    for a in kw1:   
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start():match.end()+interval]) is not None:# found 2nd kw
                    matches.append(match)
                    matches.append([i for i in re.finditer(b, line[match.start():match.end()+interval])])

    return matches # NOT FOUND

def sit2loc(line):
    kw1 = ['營業', '營運', '年', '經營']
    kw2 = ['計畫概要', '計劃概要', '計畫概述', '計劃概述', '計畫', '計劃', '方針']
    matches = list()
    interval = 10
    # 只找後面
    for a in kw1:       
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start():match.end()+interval]) is not None:# found 2nd kw
#                     print(match, line[match.start():match.end()+interval])
                    if sit2locexclude1(line[match.start():]):
                        continue
                    else:
                        matches.append(match)
                        matches.append([i for i in re.finditer(b, line[match.start():match.end()+interval])])
    return matches

def sit2locexclude1(line):
    kw1 = ['實施']
    kw2 = ['結果', '成果']
    matches = list()
    interval = 20
    # 只找後面
    for a in kw1:   
        kw_1st = re.finditer(a,line[:interval])
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[:interval]) is not None:# found 2nd kw
                    return 1

    return 0 # NOT FOUND

def sit3loc(line):
    kw1 = ['未來', '願景']
    kw2 = ['公司發展策略', '公司發展','發展','展望', '策略', '目標', '方向', '計畫', '計劃']
    matches = list()
    interval_b = 5
    interval_a = 10
    # 前後
    for a in kw1:
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start()-interval_b:match.end()+interval_a]) is not None:# found 2nd kw
                    matches.append(match)
                    matches.append([i for i in re.finditer(b, line[match.start()-interval_b:match.end()+interval_a])])
    return matches

def sit4loc(line):
    kw1 = ['外部']
    kw2 = ['競爭環境、法規環境','競爭環境','環境', '影響']
    matches = list()
    interval = 20
    # 前後
    for a in kw1:
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start()-interval:match.end()+interval]) is not None:# found 2nd kw
                    matches.append(match)
                    matches.append([i for i in re.finditer(b, line[match.start()-interval:match.end()+interval])])
    return matches

In [9]:
def sequence_speculate(paragraph):# 推測段落的編寫順序
    # 將段落集排序，list內容drop掉    
    paragraph = drop_duplicated_keyword(paragraph)
    
    global c1
    global c2
    global c3
    global c4
    # 如果以中位數來區分
#     print(paragraph)
#     from statistics import median
#     lmedian1 = median(paragraph['sit1'])
#     lmedian2 = median(paragraph['sit2'])
#     lmedian3 = median(paragraph['sit3'])
#     lmedian4 = median(paragraph['sit4'])
    
#     d = {'1':lmedian1,'2':lmedian2,'3':lmedian3,'4':lmedian4}
#     print(d)
#     seq = ''
#     for item in sorted(d.items(),key=lambda t:t[1], reverse=False):
#         seq+=item[0]
#     return seq
    
    # 如果以大數法則來區分
    l1 = len(paragraph['sit1'])
    l2 = len(paragraph['sit2'])
    l3 = len(paragraph['sit3'])
    l4 = len(paragraph['sit4'])    
    lmax = max(l1,l2,l3,l4)

    combinations = list()
    for i in range(lmax):
        if i<l1:
            c1 = paragraph['sit1'][i]
        else:
            c1 = -1
        if i<l2:
            c2 = paragraph['sit2'][i]
        else:
            c2 = -1
        if i<l3:
            c3 = paragraph['sit3'][i]
        else:
            c3 = -1
        if i<l4:
            c4 = paragraph['sit4'][i]
        else:
            c4 = -1
        
        d = {'1':c1,'2':c2,'3':c3,'4':c4}
        combinations.append(collections.OrderedDict(sorted(d.items(),key=lambda t:t[1])))
    
    ctypes = {}
    for combin in combinations:
#         print(combin)
        r = ''
        for k,v in combin.items():
            r += k
        if r in ctypes:
            ctypes[r]+=1
        else:
            ctypes[r]=1
    return sorted(ctypes.items(),key=lambda t:t[1], reverse=True)[0][0]






def title_discrimination(c1,c2,c3,c4):
    cmax = max(c1,c2,c3,c4)
    cmin = min(c1,c2,c3,c4)
    cavg = (c1+c2+c3+c4)/4
    
    # 重新排序，不管情況重小排到大
    d = {'1':c1, '2':c2, '3':c3, '4':c4}
    d = sorted(d.items(),key=lambda t:t[1])
    seq=''
    for item in d:
        seq +=item[0]
#     print(seq)
    #只要含有-1就退出
    if any([c1==-1, c2==-1, c3==-1, c4==-1]):
        return -1,seq
    # check full dist -->too short return 0
    if cmax-cmin<100:
        return 1,seq
    else:
        # check ---1-------2-3-4---   
        if (d[3][1] - d[1][1])<50: 
            return 2,seq
        # check ---1-2-3-------4---  
        elif (d[2][1] - d[0][1])<50: 
            return 3,seq
        # check ---1-2-----3---4---  
        elif (d[1][1] - d[0][1])<50: 
            return 4,seq
        # check ---1---2-----3-4---
        elif (d[3][1] - d[2][1])<50: 
            return 5,seq
        # check ---1----2-3----4---
        elif (d[2][1] - d[1][1])<50: 
            return 6,seq
        # else  ---1---2---3---4---
        else: 
            return 0,seq

In [10]:
def cut_paragraph_by_sequence(paragraph, sequence='1234'):
    # 將段落集排序，list內容drop掉    
    paragraph = drop_duplicated_keyword(paragraph)

    # 定義location 無序        
    maxlocation = {
        '1':len(paragraph['sit1']),
        '2':len(paragraph['sit2']),
        '3':len(paragraph['sit3']),
        '4':len(paragraph['sit4'])        
    }
    currentlocation = {'1':0,'2':0,'3':0,'4':0}
    classlocation = {
        '1':paragraph['sit1'][currentlocation['1']],
        '2':paragraph['sit2'][currentlocation['2']],
        '3':paragraph['sit3'][currentlocation['3']],
        '4':paragraph['sit4'][currentlocation['4']]
    }
#     print(classlocation)
            
    # 有序
    isTitle, seqs = title_discrimination(classlocation[sequence[0]],classlocation[sequence[1]],classlocation[sequence[2]],classlocation[sequence[3]])
    status = True
    exception = ''
    while isTitle!=0 and isTitle!=-1 and status:
#         print('maxlocation', maxlocation)
#         print('currentlocation', currentlocation)
#         print(isTitle, seqs)
#         print('before_classlocation', classlocation)
        if all([currentlocation['1']>=maxlocation['1'],
               currentlocation['2']>=maxlocation['2'],
               currentlocation['3']>=maxlocation['3'],
               currentlocation['4']>=maxlocation['4'] ]):
            exception = 'max limit reached'
            break
        if isTitle==1: #全改
            for i in range(4): # 改後面三個
                tempint = int(seqs[i])-1
                currentlocation[sequence[tempint]] +=1
                if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                    classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                else:
                    classlocation[sequence[tempint]] = -1
                    status=False
                    exception = '~~list out of range '+str(isTitle)
                    break
        elif isTitle==2:
            for i in range(4): # 改後面三個
                tempint = int(seqs[i])-1
                if i==0: continue
                else:
                    currentlocation[sequence[tempint]] +=1
                    if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                        classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                    else:
                        classlocation[sequence[tempint]] = -1
                        status=False
                        exception = '~~list out of range '+str(isTitle)
                        break

        elif isTitle==3:
            for i in range(3): # 改前面三個
                tempint = int(seqs[i])-1
                currentlocation[sequence[tempint]] +=1
                if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                    classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                else:
                    classlocation[sequence[tempint]] = -1
                    status=False
                    exception = '~~list out of range '+str(isTitle)
                    break

        elif isTitle==4:
            for i in range(2): # 改前面兩個
                tempint = int(seqs[i])-1
                currentlocation[sequence[tempint]] +=1
                if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                    classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                else:
                    classlocation[sequence[tempint]] = -1
                    status=False
                    exception = '~~list out of range '+str(isTitle)
                    break

        elif isTitle==5:
            for i in range(4): # 改後面兩個
                tempint = int(seqs[i])-1
                if i==0 or i==1: continue
                else:
                    currentlocation[sequence[tempint]] +=1
                    if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                        classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                    else:
                        classlocation[sequence[tempint]] = -1
                        status=False
                        exception = '~~list out of range '+str(isTitle)
                        break
        elif isTitle==6:
            for i in range(3): # 改中間兩個
                tempint = int(seqs[i])-1
                if i==0: continue
                else:
                    currentlocation[sequence[tempint]] +=1
                    if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                        classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                    else:
                        classlocation[sequence[tempint]] = -1
                        status=False
                        exception = '~~list out of range '+str(isTitle)
                        break
#         print('after_classlocation', classlocation)
        isTitle, seqs = title_discrimination(classlocation[sequence[0]],classlocation[sequence[1]],classlocation[sequence[2]],classlocation[sequence[3]])
#         print(isTitle, seqs,exception)
#         print('--------')

    # 檢查順序性
    # 1,2,3,4段具順序性
    # 如果兩個數其中一個-1就跳過
    while classlocation[sequence[1]]<classlocation[sequence[0]] and classlocation[sequence[1]]!=-1 and classlocation[sequence[0]] !=-1:
        currentlocation[sequence[1]] +=1
        if currentlocation[sequence[1]]<maxlocation[sequence[1]]:
            classlocation[sequence[1]] = paragraph['sit'+sequence[1]][currentlocation[sequence[1]]]
        else: break
    while classlocation[sequence[2]]<classlocation[sequence[1]] and classlocation[sequence[2]]!=-1 and classlocation[sequence[1]] !=-1:
        currentlocation[sequence[2]] +=1
        if currentlocation[sequence[2]]<maxlocation[sequence[2]]:
            classlocation[sequence[2]] = paragraph['sit'+sequence[2]][currentlocation[sequence[2]]]
        else: break
    while classlocation[sequence[3]]<classlocation[sequence[2]] and classlocation[sequence[3]]!=-1 and classlocation[sequence[2]] !=-1:
        currentlocation[sequence[3]] +=1
        if currentlocation[sequence[3]]<maxlocation[sequence[3]]:
            classlocation[sequence[3]] = paragraph['sit'+sequence[3]][currentlocation[sequence[3]]]
        else: break
    
    
    return classlocation, exception


In [11]:
def letter_classification(paragraph):
    classification = {'c1':0, 'c2':0, 'c3':0, 'c4':0}
    
    if len(paragraph['sit1'])!=0:
        classification['c1'] = 1
    if len(paragraph['sit2'])!=0:
        classification['c2'] = 1
    if len(paragraph['sit3'])!=0:
        classification['c3'] = 1
    if len(paragraph['sit4'])!=0:
        classification['c4'] = 1

    score=0
    for k,v in classification.items():
        score+=v
         
    return classification, score

def location_dist_too_short(loc, total, fseq):
    # if distance between two paragraph is too short return 1
    # else return 0
    count=0
    for index in fseq:
        if count==0:
            pre = loc[index]
            count+=1
            continue
        else:
            dist=loc[index]-pre
            if dist<100:
                return 1
            pre = loc[index]
            count+=1
    
    if total-pre<100:
        return 1
    
    return 0
        

In [12]:
def dump_paragraph(_obj):                  #_obj = paragrah是dict
    # 將match物件轉換為dict與list的組合
    
    if type(_obj) ==type(dict()):
        obj = _obj.copy()  # 為避免傳址問題, 但遇到dict以外物件會出現錯誤
        for k,v in obj.items():
            templist = list()
            for i in v:
                if type(i)==type(list()):
                    templist2 = list()
                    for i2 in i:
                        templist2.append({'key':i2.group(),'start':i2.start(),'end':i2.end()})
                    templist.append(templist2)
                else:
                    templist.append({'key':i.group(),'start':i.start(),'end':i.end()})
            obj[k] = templist
    else:
        obj = str(_obj)
    return obj

def drop_duplicated_keyword(paragraph):
    # 將段落集排序，list內容drop掉    
    for k,v in paragraph.items():
        templist = list()
        for items in v:
            if type(items)==type(list()):
                pass
            else:
                templist.append(items['start'])
        paragraph[k] = sorted(list(set(templist)))
        
#     print(paragraph)
    return paragraph

In [15]:
df['text'] = ''

# 開始分類每個檔案
count=0
pbar = tqdm(total=len(df))

for label, row in df.iterrows():  
    count+=1
    l=p=s=f=t=e=''
    
    try:
        file = row['inputpath']
        l,p,sc,se,f,t,e = cut_paragraph(file,row['startpage']-1,row['endpage']-1)
        #t= cut_paragraph(file,row['startpage']-1,row['endpage'])
    except Exception as ex:
        l = (-1,-1,-1,-1)
        e = str(ex)

    
    df.loc[label, 'exception'] = str(e)
    df.loc[label,'location'] = json.dumps(l)
    df.loc[label,'text'] = t
    df.loc[label,'paragraph'] = json.dumps(dump_paragraph(p))
#     df.loc[label, 'score'] = sc
#     df.loc[label,'sequence_projected'] = se
    df.loc[label,'sequence_final'] = f

    pbar.update(1)
    
#     if count==30:
#         break
pbar.close()


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:08<00:00,  6.88s/it]


In [16]:
df

,co_id,year,ext,filename,isedible,startpage,endpage,inputpath,filename_folder,text,exception,location,paragraph,sequence_final
1,1102.0,2020,pdf,2020_1102_20210625F04_20211212_154258.pdf,True,5,11,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1102/20...,2020_1102_20210625F04_20211212_154258.pdf,,list index out of range,"[-1, -1, -1, -1]","""""",
2,1103.0,2019,pdf,2019_1103_20200622F04_20211212_155216.pdf,True,4,5,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1103/20...,2019_1103_20200622F04_20211212_155216.pdf,,list index out of range,"[-1, -1, -1, -1]","""""",
4,1108.0,2019,pdf,2019_1108_20200618F04_20211212_155632.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1108/20...,2019_1108_20200618F04_20211212_155632.pdf,,list index out of range,"[-1, -1, -1, -1]","""""",
5,1108.0,2020,pdf,2020_1108_20210616F04_20211212_155819.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1108/20...,2020_1108_20210616F04_20211212_155819.pdf,,list index out of range,"[-1, -1, -1, -1]","""""",
6,1109.0,2019,pdf,2019_1109_20200623F04_20211212_155859.pdf,True,5,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1109/20...,2019_1109_20200623F04_20211212_155859.pdf,,list index out of range,"[-1, -1, -1, -1]","""""",
7,1109.0,2020,pdf,2020_1109_20210624F04_20211212_155958.pdf,True,5,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1109/20...,2020_1109_20210624F04_20211212_155958.pdf,,list index out of range,"[-1, -1, -1, -1]","""""",
8,1110.0,2019,pdf,2019_1110_20200620F04_20211212_160051.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1110/20...,2019_1110_20200620F04_20211212_160051.pdf,,list index out of range,"[-1, -1, -1, -1]","""""",
9,1110.0,2020,pdf,2020_1110_20210625F04_20211212_160217.pdf,True,5,9,C:\Users\DC\辰瑜的論文\年報108-109年\M1100水泥工業\1110/20...,2020_1110_20210625F04_20211212_160217.pdf,,list index out of range,"[-1, -1, -1, -1]","""""",
15,1216.0,2019,pdf,2019_1216_20200624F04_20211212_163421.pdf,True,6,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1200食品工業\1216/20...,2019_1216_20200624F04_20211212_163421.pdf,,list index out of range,"[-1, -1, -1, -1]","""""",
16,1216.0,2020,pdf,2020_1216_20210623F04_20211212_163450.pdf,True,6,10,C:\Users\DC\辰瑜的論文\年報108-109年\M1200食品工業\1216/20...,2020_1216_20210623F04_20211212_163450.pdf,,list index out of range,"[-1, -1, -1, -1]","""""",
